In [32]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image, sequence
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Dense, Convolution2D, Dropout, LSTM, SimpleRNN,TimeDistributed, Embedding, Bidirectional, Activation, RepeatVector, concatenate, Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Nadam, RMSprop

from tensorflow.lite.experimental.examples.lstm.rnn import bidirectional_dynamic_rnn


In [33]:
input_train = [[[0],[0],[1],[2],[3]],[[2],[3],[4],[5],[6]],[[3],[4],[5],[6],[7]],[[4],[5],[6],[7],[8]],[[5],[6],[7],[8],[9]]]
output_train = [4,7,8,9,10]

input_train1 = [[[1]]]
output_train1=[1]

input_np1 = np.array(input_train1)
output_np1 = np.array(output_train1)

input_np = np.array(input_train)
output_np = np.array(output_train)

input_train = input_np.reshape(input_np.shape[0],5,input_np.shape[2])
output_train = output_np.reshape(output_np.shape[0],1,)

print(input_train.shape)

(5, 5, 1)


In [34]:
#tf.reset_default_graph()

model = Sequential()

model.add(Dense(20, input_shape=(input_train.shape[1], input_train.shape[2])))

model.add(LSTM(units=128, input_shape=(input_train.shape[1],20), activation="relu"))

model.add(Dense(11))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

model.summary()

model.fit(input_np, output_np, batch_size=2, epochs=100)

model.save_weights("./new_lstm.h5")

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 5, 20)             40        
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               76288     
_________________________________________________________________
dense_10 (Dense)             (None, 11)                1419      
_________________________________________________________________
activation_5 (Activation)    (None, 11)                0         
Total params: 77,747
Trainable params: 77,747
Non-trainable params: 0
_________________________________________________________________
Train on 5 samples
Epoch 1/100
5/5 [==============================] - 1s 180ms/sample - loss: 2.7039 - accuracy: 0.0000e+00
Epoch 2/100
5/5 [==============================] - 0s 4ms/sample - loss: 2.2968 - accuracy: 0.2000
Epoch 3/100
5/5 [========

5/5 [==============================] - 0s 4ms/sample - loss: 0.5272 - accuracy: 1.0000
Epoch 74/100
5/5 [==============================] - 0s 4ms/sample - loss: 0.6853 - accuracy: 0.8000
Epoch 75/100
5/5 [==============================] - 0s 3ms/sample - loss: 0.6823 - accuracy: 0.6000
Epoch 76/100
5/5 [==============================] - 0s 4ms/sample - loss: 0.5115 - accuracy: 0.6000
Epoch 77/100
5/5 [==============================] - 0s 4ms/sample - loss: 0.5163 - accuracy: 0.6000
Epoch 78/100
5/5 [==============================] - 0s 4ms/sample - loss: 0.5191 - accuracy: 1.0000
Epoch 79/100
5/5 [==============================] - 0s 6ms/sample - loss: 0.6127 - accuracy: 0.8000
Epoch 80/100
5/5 [==============================] - 0s 4ms/sample - loss: 0.6093 - accuracy: 0.8000
Epoch 81/100
5/5 [==============================] - 0s 4ms/sample - loss: 0.4426 - accuracy: 1.0000
Epoch 82/100
5/5 [==============================] - 0s 4ms/sample - loss: 0.3926 - accuracy: 0.8000
Epoch 83/100


In [35]:
### convert language model to tflite

converter = tf.lite.TFLiteConverter.from_keras_model(model)

# THIS FLAG NEED TO BE SET WITH EXPERIMENTAL API 
# LSTM using keras(tensorflow v2) cannot be converted directly but using experimental api we can convert them

converter.experimental_new_converter = True 

converter.target_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                        tf.lite.OpsSet.SELECT_TF_OPS]

tflite_model = converter.convert()

open("lstm_new.tflite", "wb").write(tflite_model)

327348

In [37]:
### doing inference of tflite model

import numpy as np

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="converted_lstm_model_v1.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


input_data = np.array([[[1],[2],[3],[4],[5]]], dtype=np.float32)

print("input tensor:")
print(input_data)

interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.

output_data = interpreter.get_tensor(output_details[0]['index'])
print("output tensor:")
print(output_data)

print("\nprediction : "+str(np.argmax(output_data)) )

input tensor:
[[[1.]
  [2.]
  [3.]
  [4.]
  [5.]]]
output tensor:
[[2.1813606e-04 4.8192262e-04 1.3984852e-04 2.1053046e-07 4.7893552e-04
  4.3313697e-04 7.6579833e-01 2.1344906e-01 1.8342072e-02 6.1391271e-04
  4.4405322e-05]]

prediction : 6
